In [55]:
import os
import sys
import chromadb
from pprint import pprint
from sklearn.metrics import recall_score

sys.path.append("../")

from utils.parse_arxiv import load_json

In [5]:
res_dict = load_json("../data/arxiv/arxiv_workloads/res/k1000_gk500_pn50_n20.json")

Loaded from ../data/arxiv/arxiv_workloads/res/k1000_gk500_pn50_n20.json


In [28]:
one_res = res_dict['0']

one_res.keys()

tmp = [v['arxiv_abstract'] for k, v in res_dict.items()]

In [32]:
# tmp[0]

In [7]:
recall_score([1,2,3, 4], [1,2,104, 4], average="micro")

0.75

In [8]:
a = recall_score(one_res['arxiv_abstract'], one_res['arxiv_title'], average="micro")
a

0.005

In [9]:
b = recall_score(one_res['arxiv_abstract'], one_res['hybrid'], average="micro")
b

0.004

In [10]:
c = recall_score(one_res['arxiv_abstract'], one_res['weighted_hybrid'], average="micro")
c

0.005

In [20]:
def compute_percent_include(gt, pred, normalize=True):
    """
    Compute the percentage of ground truth that is included in the prediction

    Note: len(gt) and len(pred) doesn't have to match, but when normalize=True, we used the short length for normalization
    
    args: 
        - gt: list
        - pred: list
        - normalize: bool
    """
    # TP
    ret = sum([1 for p in pred if p in gt])
    if normalize:
        len_nor = min(len(gt), len(pred))
        ret = ret / len_nor
    return ret

In [24]:
compute_percent_include(one_res['arxiv_abstract'], one_res['arxiv_title'][-500:], True)

0.42

# Dist 

In [51]:
import chromadb

def get_ids_embed_dict(db_res_obj):
    ids_embed_dict = {}
    for i, (id, embed) in enumerate(zip(db_res_obj['ids'], db_res_obj['embeddings'])):
        ids_embed_dict[str(id)] = embed
    return ids_embed_dict

def stack_embed_inorder(res_id_list, ids_embed_dict, fill=True, verbose=False):
    embed_list = []
    if len(res_id_list) != len(ids_embed_dict):
        print("size mismatch, {} vs {}".format(len(res_id_list), len(ids_embed_dict)))
        if np.abs( len(res_id_list) - len(ids_embed_dict) ) > 1:
            # if the size mismatch is too large, return False
            return False

    for i, id in enumerate(res_id_list):
        id = str(id)
        if id not in ids_embed_dict:
            if verbose: print(i, id)
            if fill:
                id = res_id_list[i-1]
            else:
                return False
        embed_list.append(ids_embed_dict[id])
    return np.array(embed_list)

def compute_distance_metrics(gt, pred, gt_collection):
    gt_results = gt_collection.get(ids=gt, include=["embeddings"])
    pred_results = gt_collection.get(ids=pred, include=["embeddings"])
    
    # Cosine similarity
    gt_id_embed_dict = get_ids_embed_dict(gt_results)
    pred_id_embed_dict = get_ids_embed_dict(pred_results)
    gt_results = stack_embed_inorder(gt, gt_id_embed_dict)
    pred_results = stack_embed_inorder(pred, pred_id_embed_dict)
    if gt_results is False:
        print("size mismatch for gt")
        return None

    if pred_results is False:
        print("size mismatch for pred")
        return None

    distances = [cosine_similarity(g, p) for g, p in zip(gt_results, pred_results)]
    return np.mean(distances)

In [36]:
import pickle

file = open("../data/arxiv/filtered_data.pickle", "rb")
data = pickle.load(file)
file.close()

In [40]:
data[data['id']=="id_1503.00189"]

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,cat_freq,journal_freq,date_freq
602903,id_1503.00189,Stefano Pirandola,"Shabir Barzanjeh, Saikat Guha, Christian Weedb...",Microwave Quantum Illumination,Main Letter. See arXiv:1410.4008 for an extend...,"Phys. Rev. Lett. 114, 080503 (2015)",10.1103/PhysRevLett.114.080503,None,quant-ph cond-mat.other physics.ins-det physic...,http://arxiv.org/licenses/nonexclusive-distrib...,Quantum illumination is a quantum-optical se...,"[{'version': 'v1', 'created': 'Sat, 28 Feb 201...",2015-03-03,"[[Barzanjeh, Shabir, ], [Guha, Saikat, ], [Wee...",2.0,2.0,503.0


In [47]:
def print_arxiv_dp(df_row):
    print(df_row['id'].values)
    print(df_row['title'].values)
    print(df_row['abstract'].values)

In [61]:
print_arxiv_dp(data[data['id']=="id_1208.2808"])

['id_1208.2808']
['Analysis of a Statistical Hypothesis Based Learning Mechanism for Faster\n  crawling']
['  The growth of world-wide-web (WWW) spreads its wings from an intangible\nquantities of web-pages to a gigantic hub of web information which gradually\nincreases the complexity of crawling process in a search engine. A search\nengine handles a lot of queries from various parts of this world, and the\nanswers of it solely depend on the knowledge that it gathers by means of\ncrawling. The information sharing becomes a most common habit of the society,\nand it is done by means of publishing structured, semi-structured and\nunstructured resources on the web. This social practice leads to an exponential\ngrowth of web-resource, and hence it became essential to crawl for continuous\nupdating of web-knowledge and modification of several existing resources in any\nsituation. In this paper one statistical hypothesis based learning mechanism is\nincorporated for learning the behavior of c

In [57]:
chroma_client = chromadb.PersistentClient(path="../data/chroma_dbs/")
pprint(chroma_client.list_collections())
collection = chroma_client.get_collection(name="arxiv_abstract")

[Collection(name=arxiv_title), Collection(name=arxiv_abstract)]


In [59]:
collection.get(ids="id_1503.00189")

collection.get(ids="id_1311.5047")

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

In [63]:
for i in range(0, 13, 10):
    print(i)

0
10


In [71]:
data[data['id']=="id_1208.2808"]['abstract'].values[0][:2]=="  "

True

In [79]:
abstract_str = data[data['id']=="id_1208.2808"]['title'].values[0]
abstract_str = ' '.join(abstract_str.split())
abstract_str

'Analysis of a Statistical Hypothesis Based Learning Mechanism for Faster crawling'

In [74]:
abstract_str

'The growth of world-wide-web (WWW) spreads its wings from an intangible quantities of web-pages to a gigantic hub of web information which gradually increases the complexity of crawling process in a search engine. A search engine handles a lot of queries from various parts of this world, and the answers of it solely depend on the knowledge that it gathers by means of crawling. The information sharing becomes a most common habit of the society, and it is done by means of publishing structured, semi-structured and unstructured resources on the web. This social practice leads to an exponential growth of web-resource, and hence it became essential to crawl for continuous updating of web-knowledge and modification of several existing resources in any situation. In this paper one statistical hypothesis based learning mechanism is incorporated for learning the behavior of crawling speed in different environment of network, and for intelligently control of the speed of crawler. The scaling te

In [77]:
"as s as as as das d\nasd\n".split()

['as', 's', 'as', 'as', 'as', 'das', 'd', 'asd']